In [1]:
import pystan
import numpy as np
import pandas as pd
import pyper as pr
import matplotlib.pyplot as plt
%matplotlib inline
# https://pystan.readthedocs.io/en/latest/api.html
# http://tjo.hatenablog.com/entry/2014/05/01/190346

# extract data from DB to DataFrame
- ここでDataFrameにデータ入れる
- user_id, c_i1, c_i2, c_i3,,,

In [2]:
datum = np.loadtxt("women.tsv", delimiter="\t", usecols=(1, 2), skiprows=1)
weights = np.transpose(datum)[0]
heights = np.transpose(datum)[1]
y = np.array([0,0,0,0,0,0,0,1,1,1,1,1,1,1,1])

In [3]:
df = pd.read_csv("women.tsv", sep="\t").reset_index().rename(columns={"level_0":"id", "level_1":"weight", "id  height  weight":"height"}).set_index("id")
df

,weight,height
id,,
1,58,115
2,59,117
3,60,120
4,61,123
5,62,126
6,63,129
7,64,132
8,65,135
9,66,139


In [4]:
df = df.assign(weight2=lambda d: d.weight*2).assign(height2=lambda d: d.height-10).rename(columns={"weight":"w", "height":"h", "weight2":"w2", "height2":"h2"})
df

,w,h,w2,h2
id,,,,
1,58,115,116,105
2,59,117,118,107
3,60,120,120,110
4,61,123,122,113
5,62,126,124,116
6,63,129,126,119
7,64,132,128,122
8,65,135,130,125
9,66,139,132,129


In [5]:
df2 = df[["w", "h"]]
df2

,w,h
id,,
1,58,115
2,59,117
3,60,120
4,61,123
5,62,126
6,63,129
7,64,132
8,65,135
9,66,139


In [6]:
df3 = df[["w", "h", "w2"]]
df3

,w,h,w2
id,,,
1,58,115,116
2,59,117,118
3,60,120,120
4,61,123,122
5,62,126,124
6,63,129,126
7,64,132,128
8,65,135,130
9,66,139,132


In [7]:
#custom
model="""
data {
  int<lower=0> N;
  int<lower=0> D1;
  int<lower=0> D2;
  int<lower=0> D3;
  vector[D1] X1[N];
  vector[D2] X2[N];
  vector[D3] X3[N];
  int<lower=0,upper=1> Y[N];
}
parameters {
  real alpha;
  vector[D1] beta1;
  vector[D2] beta2;
  vector[D3] beta3;
}
transformed parameters {
  vector[N] p;
  for (n in 1:N) {
    vector[D1] p1 = X1[n] .* beta1;
    vector[D2] p2 = X2[n] .* beta2;
    vector[D3] p3 = X3[n] .* beta3;
    for (i in 1:D1)
      p[n] =+ p1[i];
    for (i in 1:D2)
      p[n] =+ p2[i];
    for (i in 1:D3)
      p[n] =+ p3[i];
    p[n] =+ alpha;
  }
}
model {
  alpha ~ normal(0, 1.0e+4);
  beta1 ~ normal(0, 1.0e+4);
  beta2 ~ normal(0, 1.0e+4);
  beta3 ~ normal(0, 1.0e+4);
  Y ~ bernoulli_logit(inv_logit(p));
  //for (n in 1:N)
    //Y[n] ~ bernoulli(inv_logit(p[n]));
    //Y[n] ~ bernoulli_logit(p[n]);
}
"""

In [104]:
%time fit = pystan.stan(model_code=model, data=dict(N=df.shape[0], D1=df.shape[1], D2=df2.shape[1], D3=df3.shape[1], X1=df, X2=df2, X3=df3, Y=y, chains=4), iter=1000)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_6526b6ea17f13177d448a04e748e1b52 NOW.


CPU times: user 1.35 s, sys: 93.1 ms, total: 1.44 s
Wall time: 44.1 s


In [94]:
%time fit

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 9.06 µs


Inference for Stan model: anon_model_9eece719ffb386f9c64461df5742c0ac.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

           mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
alpha      0.15    0.01   0.53  -0.89  -0.22   0.14    0.5   1.18   2000    1.0
beta1[0] 246.76   217.4 9722.2 -1.9e4  -6397 386.31 6911.1  1.9e4   2000    1.0
beta1[1] -71.47  220.83 9875.9 -1.9e4  -6728  31.38 6672.1  1.9e4   2000    1.0
beta1[2]  -8.48  238.19  1.1e4 -2.1e4  -7235  23.73 7090.3  2.2e4   2000    1.0
beta1[3] -158.9  224.85  1.0e4 -2.0e4  -6957 -13.83 6606.3  1.9e4   2000    1.0
beta2[0]  11.34   229.3  1.0e4 -2.0e4  -7196 294.02 7455.6  1.9e4   2000    1.0
beta2[1]  65.64  232.74  1.0e4 -2.0e4  -7170 -106.3 7486.3  2.0e4   2000    1.0
beta3[0] -11.91  225.06  1.0e4 -2.0e4  -6662 -50.59 6649.3  2.0e4   2000    1.0
beta3[1] -470.2  223.93  1.0e4 -2.0e4  -7393 -413.8 6460.8  1.9e4   2000    1.0
beta3[2] -122

In [ ]:
fit.plot()

In [13]:
r = pr.R(use_pandas='True', use_numpy='True')
r.assign('df', df)

In [15]:
print(r("summary(df)"))

try({summary(df)})
       w              h               w2            h2       
 Min.   :58.0   Min.   :115.0   Min.   :116   Min.   :105.0  
 1st Qu.:61.5   1st Qu.:124.5   1st Qu.:123   1st Qu.:114.5  
 Median :65.0   Median :135.0   Median :130   Median :125.0  
 Mean   :65.0   Mean   :136.7   Mean   :130   Mean   :126.7  
 3rd Qu.:68.5   3rd Qu.:148.0   3rd Qu.:137   3rd Qu.:138.0  
 Max.   :72.0   Max.   :164.0   Max.   :144   Max.   :154.0  

